# Atlas: Your Travel Planner

Time for an exciting use case. Now is the time to use all the knowledge we have gathered so far to build a complete AI Agent ourselves.

## Plan of Attack:

1. Import libraries
2. Define our tools
3. Define our tools schema
4. Define a prompt template
5. Call the OpenAI responses
6. Handle tool calls
7. The app logic
8. Gradio Interface
9. Deployment

## Step 1: Import libraries

In [ ]:
import os
from tavily import TavilyClient
from dotenv import load_dotenv
import json
from openai import OpenAI
from utils import function_to_tool
from IPython.display import display, Markdown
import gradio as gr

load_dotenv()

TAVILY_API_KEY = os.getenv("TAVILY_API_KEY")
if not TAVILY_API_KEY:
    raise ValueError("TAVILY_API_KEY is not set in the environment variables.")

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
if not OPENAI_API_KEY:
    raise ValueError("OPENAI_API_KEY is not set in the environment variables.")

tavily_client = TavilyClient()
openai_client = OpenAI()

You can setup your API key here: **[Tavily API Key](https://app.tavily.com/home)**

## Step 2: Define our tools

**The tools to define:**

1. Flight Search Tool
2. Hotel Search Tool

In [ ]:
query = "I want to travel to tokyo tomorrow what are the best flights?"

response = tavily_client.search(
    query=query,
    include_domains=["emirates.com","etihad.ae","skyscanner.com"]
)

print(json.dumps(response, indent=2))

In [ ]:
def flight_search(query: str) -> str:
    """
    Searches for flights based on the provided query

    Args:
        query (str): The search query for flights

    Returns:
        str: A formatted string containing the search results
    
    """

    response = tavily_client.search(
    query=query,
    include_domains=["emirates.com","etihad.ae","skyscanner.com"]
    )

    results = response.get("results", [])

    # extract items
    contents = [item.get("content", "") for item in results]

    # Format as a numbered list
    formatted_contents = "\n".join(f"{i + 1}. {content}" for i, content in enumerate(contents) if content)

    return formatted_contents

In [ ]:
print(flight_search(query))

In [ ]:
def hotel_search(query: str) -> str:
    """
    Searches for hotels based on the provided query

    Args:
        query (str): The search query for hotels

    Returns:
        str: A formatted string containing the search results
    
    """

    response = tavily_client.search(
    query=query,
    include_domains=["booking.com","airbnb.com"]
    )

    results = response.get("results", [])

    # extract items
    contents = [item.get("content", "") for item in results]

    # Format as a numbered list
    formatted_contents = "\n".join(f"{i + 1}. {content}" for i, content in enumerate(contents) if content)

    return formatted_contents

In [ ]:
query = "What are some cheap hotels in Tokyo?"

print(hotel_search(query))

## Step 3: Define our tools schema

In [ ]:
flight_tool_schema = function_to_tool(flight_search)
hotel_tool_schema = function_to_tool(hotel_search)

In [ ]:
hotel_tool_schema

## Step 4: Define a Prompt Template

Prompt templates are discussed in our [Prompt Engineering course](https://github.com/SuperDataScience-Community/prompt-engineering) specifically in the notebook for [multi-shot prompting](https://github.com/SuperDataScience-Community/prompt-engineering/blob/main/prompt-engineering-techniques/multi-shot-prompting.ipynb)

In [ ]:
# Class to define prompt template
class PromptTemplate:
    def __init__(self, template: str, input_variables: list[str]):
        self.template = template
        self.input_variables = input_variables

    def generate(self, **kwargs) -> str:
        return self.template.format(**{k: kwargs[k] for k in self.input_variables})

In [ ]:
prompt = PromptTemplate(
    template="I want to travel to {destination} from {origin} on {departure_date} and return on {return_date}. I prefer {preferences}",
    input_variables=["destination", "origin", "departure_date", "return_date", "preferences"]
)

In [ ]:
prompt.generate(destination="Tokyo", origin="New York", departure_date="2024-10-01", return_date="2024-10-10", preferences="budget airlines")

## Step 5: Call the OpenAI Responses API

In [ ]:
system_message = """
You are an AI travel planner. The user will provide all their vacation details in a single prompt. Your job is to:

1. Use the provided details to search for the best flights and hotels using your tools.
2. Summarize the results in a neatly formatted, easy-to-read itinerary.
3. Do not ask the user any questions or request clarification—just use the information given.
4. Make the itinerary clear, helpful, and a little bit fun!

Example flow:
- Receive the user's vacation details (destination, dates, preferences, etc.).
- Use your flight tool to find the best flight options.
- Use your hotel search tools to find the best options.
- Present a summary itinerary like:

---
**Your Adventure Awaits!**
- Destination: Tokyo, Japan
- Dates: 2024-10-01 to 2024-10-10

✈️ **Flight Options:**
1. [Flight details here]

Day 1:
🏨 **Hotel Options:**
1. [Hotel details here]
2. List of things to do

Day 2:
🏨 **Hotel Options:**
1. [Hotel details here]
2. List of things to do

Day 3:
🏨 **Hotel Options:**
1. [Hotel details here]
2. List of things to do

Have a fantastic trip!
---

Only return the final itinerary—no follow-up questions or conversation needed.
Always call the flight tool first, then the hotel tool. Do not give me a response until you have called both tools.
"""

In [ ]:
# user prompt

user_prompt = prompt.generate(
    destination="Tokyo",
    origin="New York",
    departure_date="2024-10-01",
    return_date="2024-10-10",
    preferences="budget airlines"
)

In [ ]:
# input list
input_list = [
    {"role": "system", "content": system_message},
    {"role": "user", "content": user_prompt}
]

In [ ]:
# response

response = openai_client.responses.create(
    model="gpt-4",
    input=input_list,
    tools=[flight_tool_schema, hotel_tool_schema],
    tool_choice="auto",
    parallel_tool_calls=False
)

if response.output[0].type == "message":
    input_list.append({"role": "assistant", "content": response.output_text})
if response.output[0].type == "function_call":
    input_list += response.output


In [ ]:
# print response
input_list

## Step 6: Handle tools calls

In [ ]:
def call_function(name, args):
    if name == "flight_search":
        return flight_search(**args)
    elif name == "hotel_search":
        return hotel_search(**args)
    else:
        raise ValueError(f"Unknown function: {name}")

In [ ]:
name = response.output[0].name
args = json.loads(response.output[0].arguments)

result = call_function(name, args)

input_list.append({
    'type': 'function_call_output',
    'call_id': response.output[0].call_id,
    'output': str(result)
})

In [ ]:
input_list

## Step 7: The App Logic

<div style="border-radius:16px;background:#2e3440;margin:1em 0;padding:1em;color:#eceff4;position:relative;box-shadow:0 6px 16px rgba(0,0,0,.4);overflow-wrap:break-word;word-break:break-word;">
  <b style="color:#88c0d0;font-size:1.25em">Info:</b>
  <span style="display:block;margin-top:.6em;padding-left:1.2em;line-height:1.6">
    In the video lectures, we saw that our Agent, Atlas was not following the example itinerary properly. This is probably due to the smaller sized model we are using. Try replacing that model with <code>gpt-4o</code>, <code>gpt-5</code> or if you want to continue with cheaper models, <code>gpt-4o-mini</code> all of which are great at tool use (with gpt-5 obviously outperforming all models in agentic use cases).
  </span>
  <div style="position:absolute;top:-.8em;left:-.8em;width:2.4em;height:2.4em;border-radius:50%;background:#88c0d0;color:#2e3440;display:flex;align-items:center;justify-content:center;font-weight:700;font-size:1.2em">💡</div>
</div>

In [ ]:
def get_response(input_list):
    response = openai_client.responses.create(
        model="gpt-4.1-nano",
        input=input_list,
        tools=[flight_tool_schema, hotel_tool_schema],
        tool_choice="auto",
        parallel_tool_calls=False
    )
    return response

In [ ]:
def trip_planner(destination, origin, departure_date, return_date, preferences):

    user_prompt = prompt.generate(
    destination=destination,
    origin=origin,
    departure_date=departure_date,
    return_date=return_date,
    preferences=preferences
    )

    input_list = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": user_prompt}
    ]

    response = get_response(input_list)

    if response.output[0].type == "function_call":
        while response.output[0].type == "function_call":
            input_list += response.output
            
            name = response.output[0].name
            args = json.loads(response.output[0].arguments)

            result = call_function(name, args)

            input_list.append({
                'type': 'function_call_output',
                'call_id': response.output[0].call_id,
                'output': str(result)
            })

            response = get_response(input_list)

    return response.output_text


In [ ]:
display(Markdown(trip_planner("Tokyo", "New York", "2024-10-01", "2024-10-10", "budget airlines")))

## Step 8: Gradio UI

In [ ]:
with gr.Blocks() as demo:
    gr.Markdown("# Atlas: Your Personal Travel Itinerary Planner")
    gr.Markdown("Plan your perfect trip with ease! Enter your travel details below, and let our AI-powered planner find the best flights and hotels for you.")
    
    with gr.Row():
        destination = gr.Textbox(label="Destination", placeholder="Enter your travel destination (e.g., Tokyo)")
        origin = gr.Textbox(label="Origin", placeholder="Enter your departure city (e.g., New York)")

    with gr.Row():
        departure_date = gr.Textbox(label="Departure Date", placeholder="Enter your departure date (e.g., 2024-10-01)")
        return_date = gr.Textbox(label="Return Date", placeholder="Enter your return date (e.g., 2024-10-10)")

    preferences = gr.Textbox(label="Preferences", placeholder="Enter any preferences (e.g., budget airlines, 4-star hotels)")
    submit_btn = gr.Button("Plan My Trip!")
    output = gr.Markdown(label="Your Itinerary")

    def on_submit(destination, origin, departure_date, return_date, preferences):
        return trip_planner(destination, origin, departure_date, return_date, preferences)
    
    submit_btn.click(on_submit, inputs=[destination, origin, departure_date, return_date, preferences], outputs=output)

demo.launch()

## Step 9: Deployment

You should first get setup on huggingface by making an account:

1. Visit the [huggingface](https://huggingface.co/) website and create an account.
2. Create an [API Key](https://huggingface.co/settings/tokens)
3. Copy the app logic into a `app.py` file and include a `requirements.txt` file inside of `community-contributions/your-name`
4. Open a new terminal
5. Activate our virtual environment
    - Windows command: `.venv\Scripts\activate`
    - Mac/Linux command: `source .venv/bin/activate`
6. cd into part1-fundementals using the command `cd part1-fundementals`
7. Run the command `gradio deploy`
8. Open the link to your deployed app

<div style="border-radius:16px;background:#3b1c1c;margin:1em 0;padding:1em;color:#eceff4;position:relative;box-shadow:0 6px 16px rgba(0,0,0,.4);overflow-wrap:break-word;word-break:break-word;">
  <b style="color:#bf616a;font-size:1.25em">Warning:</b>
  <span style="display:block;margin-top:.6em;padding-left:1.2em;line-height:1.6">
    You must add your <code>app.py</code>, <code>requirements.txt</code>, and a copy of <code>utils.py</code> inside the <code>community-contributions/your-name</code> folder.<br>
    For example: <code>community-contributions/your-name/app.py</code><br>
    Replace <code>your-name</code> with your actual name.
  </span>
  <div style="position:absolute;top:-.8em;left:-.8em;width:2.4em;height:2.4em;border-radius:50%;background:#bf616a;color:#2e3440;display:flex;align-items:center;justify-content:center;font-weight:700;font-size:1.2em">⚠️</div>
</div>